In [14]:
import math

import numpy as np
import pandas as pd

import lightgbm as lgb

from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits, BaseShuffleSplit, _validate_shuffle_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import log_loss

from sklearn.utils import check_random_state
from sklearn.utils.validation import _num_samples, check_array
from sklearn.utils.multiclass import type_of_target

import eli5
from IPython.display import display
from eli5.permutation_importance import get_score_importances
from eli5.sklearn import PermutationImportance

import matplotlib.pyplot as plt

import itertools
from tqdm.auto import tqdm

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


from colorama import Style, Fore

palette = ['#302c36', '#037d97', '#E4591E', '#C09741',
           '#EC5B6D', '#90A6B1', '#6ca957', '#D8E3E2']

blk = Style.BRIGHT + Fore.BLACK
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL

# Load Data

In [101]:
# COMP_PATH = "/kaggle/input/icr-identify-age-related-conditions"
COMP_PATH = "icr-identify-age-related-conditions"

train_df = pd.read_csv(f'{COMP_PATH}//train.csv')
test_df = pd.read_csv(f'{COMP_PATH}/test.csv')
greeks = pd.read_csv(f"{COMP_PATH}/greeks.csv")
sample_submission = pd.read_csv(f"{COMP_PATH}/sample_submission.csv")

train_df['EJ'] = train_df['EJ'].replace({'A': 0, 'B': 1})
test_df['EJ'] = test_df['EJ'].replace({'A': 0, 'B': 1})

train_df.columns = train_df.columns.str.replace(' ', '')
test_df.columns = test_df.columns.str.replace(' ', '')

# train_df.drop('Id',axis=1, inplace=True)
# train_df.fillna(train_df.median(), inplace=True)

In [13]:
from scipy import stats
features_with_outliers = [fe for fe in train_df.columns if fe not in ['BN', 'BQ', 'CW', 'EL', 'GH', 
                                                                      'GI', 'GL', 'Id', 'Class', 'EJ']]

pd.set_option('display.max_columns', 100)

for f in features_with_outliers:
    train_df[f] = train_df[f].clip(upper=train_df[f].quantile(0.99))

# Standard Scaler

In [3]:
# scaler = StandardScaler()
# new_num_cols = train_df.select_dtypes(include=['float64']).columns

# train_df[new_num_cols] = scaler.fit_transform(train_df[new_num_cols])
# test_df[new_num_cols] = scaler.transform(test_df[new_num_cols])

# Brute Force Feature Generation

Combine features in all possible ways.

In [4]:
# fi = pd.read_csv('feature_importances.csv', index_col = 'Unnamed: 0')
# fi_cols = set(fi['Feature'].head(100).values)

# perm = pd.read_csv('perm_df.csv', index_col = 'Unnamed: 0')
# perm_cols = set(perm['importance'].head(100).index)

# important_col = list(perm_cols.intersection(fi_cols))
# print(important_col)

# Denoising

In [5]:
# features = [fe for fe in train_df.columns if fe not in ['Id','CF', 'CB', 'DV', 'BR', 'DF', 'AR', 'GI', 'AY', 'GB',
#                                                         'AH', 'CW', 'CL', 'Class', 'BP']]

# for f in features:
#     train_df[f] = np.floor(train_df[f]*1000)/1000 # quality decreases no significant result for LGBM

# Log features (preserve sign)

In [6]:
# for f in features:
#     train_df[f] = np.sign(train_df[f]) * np.log1p(np.abs(train_df[f])) # no significant result for LGBM

# Feature generation

In [7]:
# features = train_df.drop(['Class', 'Id'], axis=1).columns

features = [fe for fe in train_df.columns if fe not in ['Id','Class']]

features = [fe for fe in train_df.columns if fe not in ['Id','CF', 'CB', 'DV', 'BR', 'DF', 'AR', 'GI', 'AY', 'GB',
                                                        'AH', 'CW', 'CL', 'Class', 'BP', 'AX', 'AZ', 'BD', 'BZ', 'EG', 
                                                        'EH', 'EJ', 'FC', 'FD', 'FE', 'FS', 'GF', 'GH']]

# ['AX', 'AZ', 'BD', 'BZ', 'EG', 'EH', 'EJ', 'FC', 'FD', 'FE', 'FS', 'GF', 'GH']

# selected = ['CR+DU', 'EH-FI', 'CD-DL', 'AB-FI', 'FL+GL', 'DU-EP', 'BQ+EP', 'FR+GL', 'CD-EP', 'AF-EG', 
#             'CU-DU', 'AB-CR', 'BQ+CD', 'BQ-GF', 'BQ+DI', 'CR+DH', 'CD+EL', 'CC+EH', 'DA-DU', 'AF+BQ', 
#             'CR+EH', 'BQ-DN', 'DN-EL', 'EJ+FI', 'BQ-EP', 'EE*EP', 'BQ/CU', 'AB*FR', 'EJ/FL', 'CD/CH', 
#             'CD/EP', 'CR*DU', 'AF/EG', 'CC/CD', 'AB/CH', 'BQ/CH', 'CH/DI', 'AB/DN', 'DU/EP', 'DU/EJ', 
#             'DU/GL', 'DY/EE', 'CR*CS', 'DH/DU', 'EJ*GL', 'CD/DL', 'DH/DI', 'DU*FR', 'CR*DH']

# def gen_features(features, df):
#     generated_features = pd.DataFrame()

#     for fe_a, fe_b in tqdm(itertools.combinations(features, 2), total=sum([1 for i in itertools.combinations(features, 2)])):
#         generated_features[f'{fe_a}+{fe_b}']   = df[fe_a] + df[fe_b]
#         generated_features[f'{fe_a}-{fe_b}']   = df[fe_a] - df[fe_b] 
#         generated_features[f'{fe_a}*{fe_b}']   = df[fe_a] * df[fe_b]
#         generated_features[f'{fe_a}/{fe_b}']   = df[fe_a] / df[fe_b]

# #         generated_features[f'{fe_a}_2']        = df[fe_a].pow(2)
# #         generated_features[f'{fe_b}_2']        = df[fe_b].pow(2)
# #         generated_features[f'{fe_a}*{fe_b}_2'] = df[fe_a] * df[fe_b].pow(2)
# #         generated_features[f'{fe_a}_2*{fe_b}'] = df[fe_a].pow(2) * df[fe_b]

# #         generated_features[f'{fe_a}_05'] = df[fe_a].pow(0.5)
# #         generated_features[f'{fe_b}_05'] = df[fe_b].pow(0.5)
# #         generated_features[f'{fe_a}*{fe_b}_05'] = df[fe_a] * df[fe_b].pow(0.5)
# #         generated_features[f'{fe_a}_05*{fe_b}'] = df[fe_a].pow(0.5) * df[fe_b]

# #         generated_features[f'{fe_a}_log'] = np.log(df[fe_a])
# #         generated_features[f'{fe_b}_log'] = np.log(df[fe_b])
# #         generated_features[f'{fe_a}*{fe_b}_log'] = df[fe_a] * np.log(df[fe_b])
# #         generated_features[f'{fe_a}_log*{fe_b}'] = np.log(df[fe_a]) * df[fe_b]
        
#     generated_features = generated_features[selected]
#     generated_features = pd.concat([generated_features, df[features]], axis=1)
    
#     # prevent inf
#     for g in generated_features.columns:
#         generated_features[g] = np.minimum(np.maximum(generated_features[g], -1e9), 1e9)
    
#     return generated_features

# generated_features_train = gen_features(features, train_df)
# generated_features_test = gen_features(features, test_df)

# Multilabel Stratification

In [8]:
def IterativeStratification(labels, r, random_state):
    """This function implements the Iterative Stratification algorithm described
    in the following paper:
    Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of
    Multi-Label Data. In: Gunopulos D., Hofmann T., Malerba D., Vazirgiannis M.
    (eds) Machine Learning and Knowledge Discovery in Databases. ECML PKDD
    2011. Lecture Notes in Computer Science, vol 6913. Springer, Berlin,
    Heidelberg.
    """

    n_samples = labels.shape[0]
    test_folds = np.zeros(n_samples, dtype=int)

    # Calculate the desired number of examples at each subset
    c_folds = r * n_samples

    # Calculate the desired number of examples of each label at each subset
    c_folds_labels = np.outer(r, labels.sum(axis=0))

    labels_not_processed_mask = np.ones(n_samples, dtype=bool)

    while np.any(labels_not_processed_mask):
        # Find the label with the fewest (but at least one) remaining examples,
        # breaking ties randomly
        num_labels = labels[labels_not_processed_mask].sum(axis=0)

        # Handle case where only all-zero labels are left by distributing
        # across all folds as evenly as possible (not in original algorithm but
        # mentioned in the text). (By handling this case separately, some
        # code redundancy is introduced; however, this approach allows for
        # decreased execution time when there are a relatively large number
        # of all-zero labels.)
        if num_labels.sum() == 0:
            sample_idxs = np.where(labels_not_processed_mask)[0]

            for sample_idx in sample_idxs:
                fold_idx = np.where(c_folds == c_folds.max())[0]

                if fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(fold_idx.shape[0])]

                test_folds[sample_idx] = fold_idx
                c_folds[fold_idx] -= 1

            break

        label_idx = np.where(num_labels == num_labels[np.nonzero(num_labels)].min())[0]
        if label_idx.shape[0] > 1:
            label_idx = label_idx[random_state.choice(label_idx.shape[0])]

        sample_idxs = np.where(np.logical_and(labels[:, label_idx].flatten(), labels_not_processed_mask))[0]

        for sample_idx in sample_idxs:
            # Find the subset(s) with the largest number of desired examples
            # for this label, breaking ties by considering the largest number
            # of desired examples, breaking further ties randomly
            label_folds = c_folds_labels[:, label_idx]
            fold_idx = np.where(label_folds == label_folds.max())[0]

            if fold_idx.shape[0] > 1:
                temp_fold_idx = np.where(c_folds[fold_idx] ==
                                         c_folds[fold_idx].max())[0]
                fold_idx = fold_idx[temp_fold_idx]

                if temp_fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(temp_fold_idx.shape[0])]

            test_folds[sample_idx] = fold_idx
            labels_not_processed_mask[sample_idx] = False

            # Update desired number of examples
            c_folds_labels[fold_idx, labels[sample_idx]] -= 1
            c_folds[fold_idx] -= 1

    return test_folds


class MultilabelStratifiedKFold(_BaseKFold):
    """Multilabel stratified K-Folds cross-validator
    Provides train/test indices to split multilabel data into train/test sets.
    This cross-validation object is a variation of KFold that returns
    stratified folds for multilabel data. The folds are made by preserving
    the percentage of samples for each label.
    Parameters
    ----------
    n_splits : int, default=3
        Number of folds. Must be at least 2.
    shuffle : boolean, optional
        Whether to shuffle each stratification of the data before splitting
        into batches.
    random_state : int, RandomState instance or None, optional, default=None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedKFold that only uses random_state
        when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0)
    >>> mskf.get_n_splits(X, y)
    2
    >>> print(mskf)  # doctest: +NORMALIZE_WHITESPACE
    MultilabelStratifiedKFold(n_splits=2, random_state=0, shuffle=False)
    >>> for train_index, test_index in mskf.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    Notes
    -----
    Train and test sizes may be slightly different in each fold.
    See also
    --------
    RepeatedMultilabelStratifiedKFold: Repeats Multilabel Stratified K-Fold
    n times.
    """

    def __init__(self, n_splits=3, *, shuffle=False, random_state=None):
        super(MultilabelStratifiedKFold, self).__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def _make_test_folds(self, X, y):
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(type_of_target_y))

        num_samples = y.shape[0]

        rng = check_random_state(self.random_state)
        indices = np.arange(num_samples)

        if self.shuffle:
            rng.shuffle(indices)
            y = y[indices]

        r = np.asarray([1 / self.n_splits] * self.n_splits)

        test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

        return test_folds[np.argsort(indices)]

    def _iter_test_masks(self, X=None, y=None, groups=None):
        test_folds = self._make_test_folds(X, y)
        for i in range(self.n_splits):
            yield test_folds == i

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedKFold, self).split(X, y, groups)


class RepeatedMultilabelStratifiedKFold(_RepeatedSplits):
    """Repeated Multilabel Stratified K-Fold cross validator.
    Repeats Mulilabel Stratified K-Fold n times with different randomization
    in each repetition.
    Parameters
    ----------
    n_splits : int, default=5
        Number of folds. Must be at least 2.
    n_repeats : int, default=10
        Number of times cross-validator needs to be repeated.
    random_state : None, int or RandomState, default=None
        Random state to be used to generate random state for each
        repetition as well as randomly breaking ties within the iterative
        stratification algorithm.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import RepeatedMultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> rmskf = RepeatedMultilabelStratifiedKFold(n_splits=2, n_repeats=2,
    ...     random_state=0)
    >>> for train_index, test_index in rmskf.split(X, y):
    ...     print("TRAIN:", train_index, "TEST:", test_index)
    ...     X_train, X_test = X[train_index], X[test_index]
    ...     y_train, y_test = y[train_index], y[test_index]
    ...
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    TRAIN: [0 1 4 5] TEST: [2 3 6 7]
    TRAIN: [2 3 6 7] TEST: [0 1 4 5]
    See also
    --------
    RepeatedStratifiedKFold: Repeats (Non-multilabel) Stratified K-Fold
    n times.
    """
    def __init__(self, n_splits=5, *, n_repeats=10, random_state=None):
        super(RepeatedMultilabelStratifiedKFold, self).__init__(
            MultilabelStratifiedKFold, n_repeats=n_repeats, random_state=random_state,
            n_splits=n_splits)


class MultilabelStratifiedShuffleSplit(BaseShuffleSplit):
    """Multilabel Stratified ShuffleSplit cross-validator
    Provides train/test indices to split data into train/test sets.
    This cross-validation object is a merge of MultilabelStratifiedKFold and
    ShuffleSplit, which returns stratified randomized folds for multilabel
    data. The folds are made by preserving the percentage of each label.
    Note: like the ShuffleSplit strategy, multilabel stratified random splits
    do not guarantee that all folds will be different, although this is
    still very likely for sizeable datasets.
    Parameters
    ----------
    n_splits : int, default 10
        Number of re-shuffling & splitting iterations.
    test_size : float, int, None, optional
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        complement of the train size. By default, the value is set to 0.1.
        The default will change in version 0.21. It will remain 0.1 only
        if ``train_size`` is unspecified, otherwise it will complement
        the specified ``train_size``.
    train_size : float, int, or None, default is None
        If float, should be between 0.0 and 1.0 and represent the
        proportion of the dataset to include in the train split. If
        int, represents the absolute number of train samples. If None,
        the value is automatically set to the complement of the test size.
    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedShuffleSplit that only uses
        random_state when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.5,
    ...    random_state=0)
    >>> msss.get_n_splits(X, y)
    3
    >>> print(mss)       # doctest: +ELLIPSIS
    MultilabelStratifiedShuffleSplit(n_splits=3, random_state=0, test_size=0.5,
                                     train_size=None)
    >>> for train_index, test_index in msss.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    TRAIN: [2 3 6 7] TEST: [0 1 4 5]
    TRAIN: [1 2 5 6] TEST: [0 3 4 7]
    Notes
    -----
    Train and test sizes may be slightly different from desired due to the
    preference of stratification over perfectly sized folds.
    """

    def __init__(self, n_splits=10, *, test_size="default", train_size=None,
                 random_state=None):
        super(MultilabelStratifiedShuffleSplit, self).__init__(
            n_splits=n_splits, test_size=test_size, train_size=train_size, random_state=random_state)

    def _iter_indices(self, X, y, groups=None):
        n_samples = _num_samples(X)
        y = check_array(y, ensure_2d=False, dtype=None)
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(
                    type_of_target_y))

        n_train, n_test = _validate_shuffle_split(n_samples, self.test_size,
                                                  self.train_size)

        n_samples = y.shape[0]
        rng = check_random_state(self.random_state)
        y_orig = y.copy()

        r = np.array([n_train, n_test]) / (n_train + n_test)

        for _ in range(self.n_splits):
            indices = np.arange(n_samples)
            rng.shuffle(indices)
            y = y_orig[indices]

            test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

            test_idx = test_folds[np.argsort(indices)] == 1
            test = np.where(test_idx)[0]
            train = np.where(~test_idx)[0]

            yield train, test

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedShuffleSplit, self).split(X, y, groups)

# LGBM hyperparameters tuning

In [9]:
#  train_df.isnull().sum()

In [10]:
from shaphypetune import BoostBoruta

class CFG:
    n_repeats = 4
    n_folds = 5
    num_boost_round = 10000
    seeds = [1, 42, 228, 265, 21, 8081988, 5062023, 666, 1488]

params = {
        'boosting_type':'goss',
        'learning_rate': 0.06733232950390658, 
        'n_estimators': 50000, 
        'early_stopping_round' : 100, 
        'subsample' : 0.6970532011679706,
        'colsample_bytree': 0.6055755840633003,
        'num_leaves': 6,
        'class_weight': 'balanced',
        'metric': 'none', 
        'is_unbalance': True, 
        'random_state': 8062023,
        'feature_fraction_seed': 8062023,
        'bagging_seed': 8062023,
        'max_depth': 8,
        'reg_alpha': 0.08866046540248787,  
        'reg_lambda': 1.0245261859148395e-06,
        'importance_type': 'gain'
        }

def balanced_log_loss(y_true, y_pred):

    # Nc is the number of observations
    N_1 = np.sum(y_true == 1, axis=0)
    N_0 = np.sum(y_true == 0, axis=0)

    # In order to avoid the extremes of the log function, each predicted probability 𝑝 is replaced with max(min(𝑝,1−10−15),10−15)
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)

    # balanced logarithmic loss
    loss_numerator = - (1/N_0) * np.sum((1 - y_true) * np.log(1 - y_pred)) - (1/N_1) * np.sum(y_true * np.log(y_pred))

    return loss_numerator / 2

def bll_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

def calc_log_loss_weight(y_true): 
    '''w0, w1 assign different weights to individual data points during training.'''
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    return w0, w1

def lgbm_tuning(features, permut=False, boruta=False):
    metric = balanced_log_loss
    eval_results_ = {}

    outer_cv_score = [] # store all cv scores of outer loop inference
    inner_cv_score = [] # store all cv scores of inner loop training

    perm_df_ = pd.DataFrame()
    feature_importances_ = pd.DataFrame()
    boruta_df_ = pd.DataFrame()
    
    for i in range(CFG.n_repeats):
        print(f'Repeat {blu}#{i+1}')
        
        kf = MultilabelStratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=8062023+i)

        # Stratify based on Class and Alpha (3 types of conditions)
        for fold, (train_idx, val_idx) in enumerate(kf.split(X=train_df[features], y=greeks.iloc[:,1:3]), start = 1): 
            X, y = train_df[features], train_df.Class
#             X, y = generated_features_train, train_df.Class
            # Split the dataset according to the fold indexes.
            X_train = X.iloc[train_idx]
            X_val = X.iloc[val_idx]
            y_train = y.iloc[train_idx]
            y_val = y.iloc[val_idx]

            # 20% hold-out set
            X_holdout, y_holdout = X_val, y_val

            # Create an oof array for inner loop
            oof_inner = np.zeros(len(X_train))

            X_train = X_train.reset_index(drop=True)
            y_train = y_train.reset_index(drop=True)

            cv = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=8062023+i) # Use stratifiedKfold to make life easier

            X_outer, y_outer = X_train, y_train

            models_ = [] # Used to store models trained in the inner loop.

            print(f"Outer Loop fold {fold}, Inner Loop Training with {blu}{X_train.shape[0]}{res} samples, {blu}{X_train.shape[1]}{res} features, seed = {blu}{8602023}{res}")

            for fold, (train_idx, val_idx) in enumerate(cv.split(X=X_train, y=y_train), start = 1):
                # Split the dataset according to the fold indexes.
                X_train = X_outer.iloc[train_idx]
                X_val = X_outer.iloc[val_idx]
                y_train = y_outer.iloc[train_idx]
                y_val = y_outer.iloc[val_idx]

    #             trn_w0, trn_w1 = calc_log_loss_weight(y_train)
    #             val_w0, val_w1 = calc_log_loss_weight(y_train)

    #             w_val = [y_train.map({0: trn_w0, 1: trn_w1}), y_val.map({0: val_w0, 1: val_w1})]

                eval_results_[fold]= {}

                clf = lgb.LGBMClassifier(**params)
                clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], 
                        eval_metric=bll_metric, # eval_sample_weight=w_val, 
                        early_stopping_rounds=300, verbose=-1)

                models_.append(clf)

                val_preds = clf.predict_proba(X_val)[:,1]
                oof_inner[val_idx] = val_preds

                val_score = metric(y_val, val_preds)
                best_iter = clf.best_iteration_

                print(f'Fold: {blu}{fold:>3}{res}| {metric.__name__}: {blu}{val_score:.5f}{res}'
                      f' | Best iteration: {blu}{best_iter:>4}{res}')

                # permutation importance
                if permut:
                    perm = PermutationImportance(clf, scoring=None, n_iter=1, 
                                                 random_state=42, cv=None, refit=False).fit(X_val, y_val)

                    perm_importance_df = pd.DataFrame({'importance': perm.feature_importances_}, 
                                                       index=X_val.columns).sort_index()

                    if perm_df_.shape[0] == 0:
                        perm_df_ = perm_importance_df.copy()
                    else:
                        perm_df_ += perm_importance_df

                # tree feature importance
                f_i = pd.DataFrame(sorted(zip(clf.feature_importances_, X.columns), 
                                                  reverse=True, key=lambda x: x[1]), 
                                   columns=['Value','Feature'])

                if feature_importances_.shape[0] == 0:
                    feature_importances_ = f_i.copy()
                else:

                    feature_importances_['Value'] += f_i['Value']
                    
                # Boruta SHAP importance
                if boruta:
                    model = BoostBoruta(clf, importance_type='shap_importances', train_importance=False)
                    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], 
                              eval_metric=bll_metric, early_stopping_rounds=300, verbose=-1)
                    
                    boruta_importance_df = pd.DataFrame({'importance': model.ranking_}, 
                                                         index=X_train.columns).sort_index()
                    if boruta_df_.shape[0] == 0:
                        boruta_df_ = boruta_importance_df.copy()
                    else:
                        boruta_df_ += boruta_importance_df

            mean_cv_score = metric(y_outer, oof_inner)
            print(f'{red} Inner CV score: {res} {metric.__name__}: {red}{mean_cv_score:.5f}{res}')
            print(f'{"*" * 50}\n')
            inner_cv_score.append(mean_cv_score)

            # infer holdout data using 5-fold model trained in inner loop
            preds = np.zeros(len(X_holdout))
            for model in models_:
                preds += model.predict_proba(X_holdout)[:,1]
            preds = preds / len(models_)
            cv_score = metric(y_holdout, preds)
            print(f'{red} Outer Holdout score: {res} {metric.__name__}: {red}{cv_score:.5f}{res}')
            print(f'{"*" * 50}\n')
            outer_cv_score.append(cv_score)

    print(f'{red} Inner CV avg score: {res} {metric.__name__}: {red}{np.mean(inner_cv_score):.5f}{res}')
    print(f'{"*" * 50}\n')

    print(f'{red} Outer Holdout avg score: {res} {metric.__name__}: {red}{np.mean(outer_cv_score):.5f}{res}')
    print(f'{"*" * 50}\n')
    
    if permut:
        perm_df_ = perm_df_.sort_values('importance', ascending=False)
        
    if boruta:
        boruta_df_ = boruta_df_.sort_values('importance')
                                    
    feature_importances_ = feature_importances_.sort_values('Value', ascending=False)
    
    return perm_df_, feature_importances_, boruta_df_, np.mean(inner_cv_score), np.mean(outer_cv_score)

perm_df_, feature_importances_, boruta_df_, inner_cv_score, outer_cv_score = lgbm_tuning(features, permut=False, boruta=False)

Repeat #1
Outer Loop fold 1, Inner Loop Training with 494 samples, 30 features, seed = 8602023
Fold:   1| balanced_log_loss: 0.22666 | Best iteration:  176
Fold:   2| balanced_log_loss: 0.18390 | Best iteration:  122
Fold:   3| balanced_log_loss: 0.29212 | Best iteration:  176
Fold:   4| balanced_log_loss: 0.20285 | Best iteration:  239
Fold:   5| balanced_log_loss: 0.12348 | Best iteration:  277
 Inner CV score:  balanced_log_loss: 0.20597
**************************************************

 Outer Holdout score:  balanced_log_loss: 0.16362
**************************************************

Outer Loop fold 2, Inner Loop Training with 494 samples, 30 features, seed = 8602023
Fold:   1| balanced_log_loss: 0.35666 | Best iteration:  206
Fold:   2| balanced_log_loss: 0.09546 | Best iteration:  408
Fold:   3| balanced_log_loss: 0.19520 | Best iteration:  136
Fold:   4| balanced_log_loss: 0.28382 | Best iteration:  161
Fold:   5| balanced_log_loss: 0.15390 | Best iteration:  334
 Inner CV s

Fold:   4| balanced_log_loss: 0.22011 | Best iteration:  206
Fold:   5| balanced_log_loss: 0.29011 | Best iteration:  144
 Inner CV score:  balanced_log_loss: 0.22243
**************************************************

 Outer Holdout score:  balanced_log_loss: 0.14072
**************************************************

Repeat #3
Outer Loop fold 1, Inner Loop Training with 493 samples, 30 features, seed = 8602023
Fold:   1| balanced_log_loss: 0.19742 | Best iteration:  168
Fold:   2| balanced_log_loss: 0.15782 | Best iteration:  154
Fold:   3| balanced_log_loss: 0.16116 | Best iteration:  230
Fold:   4| balanced_log_loss: 0.32973 | Best iteration:  195
Fold:   5| balanced_log_loss: 0.21102 | Best iteration:   97
 Inner CV score:  balanced_log_loss: 0.21088
**************************************************

 Outer Holdout score:  balanced_log_loss: 0.26698
**************************************************

Outer Loop fold 2, Inner Loop Training with 494 samples, 30 features, seed = 860

Fold:   1| balanced_log_loss: 0.22620 | Best iteration:  185
Fold:   2| balanced_log_loss: 0.12869 | Best iteration:  236
Fold:   3| balanced_log_loss: 0.27827 | Best iteration:  196
Fold:   4| balanced_log_loss: 0.24972 | Best iteration:  171
Fold:   5| balanced_log_loss: 0.25074 | Best iteration:  241
 Inner CV score:  balanced_log_loss: 0.22628
**************************************************

 Outer Holdout score:  balanced_log_loss: 0.12366
**************************************************

 Inner CV avg score:  balanced_log_loss: 0.20305
**************************************************

 Outer Holdout avg score:  balanced_log_loss: 0.21093
**************************************************



# Eliminate features by their score

In [11]:
# f_dict = dict()
# features = list(features)

# for i, f in tqdm(enumerate(features), total=len(features)):
#     _, __, inner_cv_score, outer_cv_score = lgbm_tuning(features[:i] + features[i+1:], permut=False)
#     f_dict[f] = (inner_cv_score, outer_cv_score)
    

Inner CV avg score:  balanced_log_loss: 0.20305
**************************************************

 Outer Holdout avg score:  balanced_log_loss: 0.21093
**************************************************

# Analyze permutation feature importance

In [12]:
# perm_df_.to_csv('perm_df.csv')
# perm_df_
# perm_cols = set(perm_df_.index[-20:])
# perm_cols

# Analyze tree gain feature importance

In [13]:
# feature_importances_.to_csv('feature_importances.csv')
# feature_importances_
# fi_cols = set(feature_importances_['Feature'].values[-20:])
# fi_cols

# Analyze BORUTA importance

In [14]:
# boruta_df_.to_csv('perm_df.csv')
# boruta_df_
# boruta_cols = set(boruta_df_.index[-20:])
# boruta_cols

# LGBM training

In [23]:
class CFG:
    n_repeats = 4
    n_folds = 5
    num_boost_round = 10000
    seeds = [1, 42, 228, 265, 21, 8081988, 5062023, 666, 1488]

params = {
        'boosting_type':'goss',
        'learning_rate': 0.06733232950390658, 
        'n_estimators': 50000, 
        'early_stopping_round' : 100, 
        'subsample' : 0.6970532011679706,
        'colsample_bytree': 0.6055755840633003,
        'num_leaves': 6,
        'class_weight': 'balanced',
        'metric': 'none', 
        'is_unbalance': True, 
        'random_state': 8062023,
        'feature_fraction_seed': 8062023,
        'bagging_seed': 8062023,
        'max_depth': 8,
        'reg_alpha': 0.08866046540248787,  
        'reg_lambda': 1.0245261859148395e-06,
        'importance_type': 'gain'
        }

def balanced_log_loss(y_true, y_pred):

    # Nc is the number of observations
    N_1 = np.sum(y_true == 1, axis=0)
    N_0 = np.sum(y_true == 0, axis=0)

    # In order to avoid the extremes of the log function, each predicted probability 𝑝 is replaced with max(min(𝑝,1−10−15),10−15)
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)

    # balanced logarithmic loss
    loss_numerator = - (1/N_0) * np.sum((1 - y_true) * np.log(1 - y_pred)) - (1/N_1) * np.sum(y_true * np.log(y_pred))

    return loss_numerator / 2

def bll_metric(y_true, y_pred):
    return 'balanced_log_loss', balanced_log_loss(y_true, y_pred), False

def lgbm_training():
    models_ = list()
    all_cv_score = list()
    weights_ = list()
    
    all_eval_results_ = dict() # used to store evaluation results for each seed
    X, y = train_df[features], train_df.Class
#     X, y = generated_features_train, train_df.Class
    
    for seed in CFG.seeds:   
        kf = MultilabelStratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=seed)
        metric = balanced_log_loss
        eval_results_ = {}     # used to store evaluation results for each fold
        oof = np.zeros(len(X))
        print(f"Training with {blu}{X.shape[1]}{res} features, seed = {blu}{seed}{res}")
        
        for fold, (fit_idx, val_idx) in enumerate(kf.split(X=train_df, y=greeks.iloc[:,1:3]), start = 1):
            # Split the dataset according to the fold indexes.
            X_train = X.iloc[fit_idx]
            X_val = X.iloc[val_idx]
            y_train = y.iloc[fit_idx]
            y_val = y.iloc[val_idx]

#             trn_w0, trn_w1 = calc_log_loss_weight(y_train)
#             val_w0, val_w1 = calc_log_loss_weight(y_val)

#             w_val = [y_train.map({0: trn_w0, 1: trn_w1}), y_val.map({0: val_w0, 1: val_w1})]

            clf = lgb.LGBMClassifier(**params)
            clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], 
                    eval_metric=bll_metric, early_stopping_rounds=300,
                    verbose=-1)

            eval_results_[fold]= clf.evals_result_
            
            val_preds = clf.predict_proba(X_val)[:,1]
            oof[val_idx] = val_preds

            val_score = balanced_log_loss(y_val, val_preds)
            best_iter = clf.best_iteration_
            
            print(f'Fold: {blu}{fold:>3}{res}| {metric.__name__}: {blu}{val_score:.5f}{res}'
                  f' | Best iteration: {blu}{best_iter:>4}{res}')

            # Stores the model
            models_.append(clf)
            
            # Stores evaluation result
            all_eval_results_[seed] = eval_results_
            
            # Store prediction weight
            weights_.append(1/clf.evals_result_['valid_1']['balanced_log_loss'][clf.best_iteration_])
            
#         oof[(oof > 0.8) & (oof < 0.9)] = 0.9
#         oof[(oof < 0.2) & (oof > 0.1)] = 0.1
        mean_cv_score = metric(y, oof)
        
        
        all_cv_score.append(mean_cv_score)
        print(f'{red}Mean{res} {metric.__name__}: {red}{mean_cv_score:.5f}')
        print(f'{"*" * 50}\n')
    
    print(f'{red}Mean{res} {metric.__name__} for all seeds: {red}{np.mean(all_cv_score):.5f}')
    return models_, eval_results_, all_eval_results_, weights_

models_, eval_results_, all_eval_results_, weights_ = lgbm_training()

Training with 30 features, seed = 1
Fold:   1| balanced_log_loss: 0.07147 | Best iteration: 1083
Fold:   2| balanced_log_loss: 0.29489 | Best iteration:  108
Fold:   3| balanced_log_loss: 0.15393 | Best iteration:  184
Fold:   4| balanced_log_loss: 0.24775 | Best iteration:  117
Fold:   5| balanced_log_loss: 0.14126 | Best iteration:  237
Mean balanced_log_loss: 0.18491
**************************************************

Training with 30 features, seed = 42
Fold:   1| balanced_log_loss: 0.30582 | Best iteration:  112
Fold:   2| balanced_log_loss: 0.14901 | Best iteration:  250
Fold:   3| balanced_log_loss: 0.08629 | Best iteration:  504
Fold:   4| balanced_log_loss: 0.14131 | Best iteration:  354
Fold:   5| balanced_log_loss: 0.17714 | Best iteration:  176
Mean balanced_log_loss: 0.17232
**************************************************

Training with 30 features, seed = 228
Fold:   1| balanced_log_loss: 0.14511 | Best iteration:  276
Fold:   2| balanced_log_loss: 0.23579 | Best itera

Mean balanced_log_loss for all seeds: 0.18615

# Model evalutation

In [16]:
# ## Model Evaluation
# metric_score_folds = pd.DataFrame.from_dict(all_eval_results_)
# fit_logloss = []
# val_logloss = []

# for seed in CFG.seeds:
#     for fold in range(1,CFG.n_folds+1):
#         fit_logloss.append(metric_score_folds[seed][fold]['training']['balanced_log_loss'])
#         val_logloss.append(metric_score_folds[seed][fold]['valid_1']['balanced_log_loss'])

# fig, axes = plt.subplots(math.ceil(CFG.n_folds*len(CFG.seeds)/CFG.n_folds), CFG.n_folds, figsize=(20, 20), dpi=150)
# ax = axes.flatten()
# for i, (f, v, m) in enumerate(zip(fit_logloss, val_logloss, models_), start = 1): 
#     sns.lineplot(f, color='#B90000', ax=ax[i-1], label='fit')
#     sns.lineplot(v, color='#048BA8', ax=ax[i-1], label='val')
#     ax[i-1].legend()
#     ax[i-1].spines['top'].set_visible(False);
#     ax[i-1].spines['right'].set_visible(False)
#     ax[i-1].set_title(f'Seed {CFG.seeds[(i-1)//CFG.n_folds]} Fold {CFG.n_folds if i%CFG.n_folds==0 else i%CFG.n_folds}', fontdict={'fontweight': 'bold'})

#     color =  ['#048BA8', palette[-3]]
#     best_iter = m.best_iteration_
#     span_range = [[0, best_iter], [best_iter + 10, best_iter + CFG.num_boost_round]]

#     for idx, sub_title in enumerate([f'Best\nIteration: {best_iter}', f'Early\n Stopping: 2000']):
#         ax[i-1].annotate(sub_title,
#                     xy=(sum(span_range[idx])/2 , 0.5),
#                     xytext=(0,0), textcoords='offset points',
#                     va="center", ha="center",
#                     color="w", fontsize=16, fontweight='bold',
#                     bbox=dict(boxstyle='round4', pad=0.4, color=color[idx], alpha=0.6))
#         ax[i-1].axvspan(span_range[idx][0]-0.4,span_range[idx][1]+0.4,  color=color[idx], alpha=0.07)

#     ax[i-1].set_xlim(0, best_iter + 20 + 2000)
#     ax[i-1].legend(bbox_to_anchor=(0.95, 1), loc='upper right', title='logloss')

# plt.tight_layout();

# Predict test

In [17]:
# test_preds = np.zeros((test_df.shape[0],2))

# for i in range(len(final_test_predictions)):
#     test_preds[:, 0] += test_weights[i] * final_test_predictions[i][:, 0]
#     test_preds[:, 1] += test_weights[i] * final_test_predictions[i][:, 1]

# test_preds /= sum(test_weights)

def predict(X):
    y = np.zeros(len(X))
    for i, model in enumerate(models_):
#         y += weights_[i] * model.predict_proba(X)[:,1]
        y += model.predict_proba(X)[:,1]
#     return y / sum(weights_)
    return y / len(models_)

predictions = predict(test_df[features])
# predictions = predict(generated_features_test)

test_df['class_1'] = predictions
test_df['class_0'] = 1 - predictions

sample_submission[['class_0', 'class_1']] = test_df[['class_0', 'class_1']]
sample_submission.to_csv(r"submission.csv", index=False)
sample_submission

,Id,class_0,class_1
0,00eed32682bb,0.590987,0.409013
1,010ebe33f668,0.590987,0.409013
2,02fa521e1838,0.590987,0.409013
3,040e15f562a2,0.590987,0.409013
4,046e85c7cc7f,0.590987,0.409013


You have a lot of resulting features. I have already identified a few important once. 